In [1]:
import warnings
warnings.simplefilter("ignore")
from sys import exit
import os.path
import cv2
import numpy as np
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory
from scipy import ndimage
from skimage import measure, io, filters, morphology
from pathlib import Path
from skimage.segmentation import watershed
import matplotlib.pyplot as plt
from math import sqrt
import csv

In [2]:
'''
from matplotlib import text
from skimage.morphology import disk
from scipy import signal
from detecta import detect_peaks
'''

'\nfrom matplotlib import text\nfrom skimage.morphology import disk\nfrom scipy import signal\nfrom detecta import detect_peaks\n'

In [3]:
'''
outL: box size when identifying molecules
splitter: amount of frames between check-ins to get new molecules
frameRate = frame rate of camera to determine time
'''
outL = int(49)
splitter = 50
frameRate = 10

'''
showimgs: shows all images for testing, set as false for just running
saveimgs: allows user to save images
saveForTest: saves additional files full of images
saveForPeaks: saves additional files for precise peak reading
'''
showimgs = False
saveimgs = True
saveForTest = False
saveForPeaks = False

enter = 13
right = 83
left = 81
esc = 27

wantGain = False
'''
Define pixel size of current camera in nm/pix-el: MD: 114, Andor: 128, Sam: 101.4
'''
pixelSize = 128
refPt = []

In [4]:
def my_convert_16U_2_8U(image):
    min_ = np.amin(image)
    max_ = np.amax(image)
    a = 255/float(max_-min_)
    b = -a*min_ 
    img8U = np.zeros(image.shape,np.uint8)
    cv2.convertScaleAbs(image,img8U,a,b)
    return img8U

In [5]:
def getBrightBox(img, point, outLength):
    shap = img.shape
    startx = int(max(0, point[1] - outLength))
    starty = int(max(0, point[0] - outLength))
    endx = int(min(shap[1], point[1] + outLength+1))
    endy = int(min(shap[0], point[0] + outLength+1))
    ml = int(min(0, point[1] - outLength))
    mr = int(max(0, point[1] + outLength + 1 - shap[1]))
    mt = int(min(0, point[0] - outLength))
    mb = int(max(0, point[0] + outLength + 1 - shap[0]))
    miss = [int(mt*-1), int(mb), int(ml*-1), int(mr)]
    brightBox = img[starty:endy,startx:endx]
    avg = brightBox.mean()
    return(brightBox, startx, starty, avg, miss)

In [6]:
def makeFolders(numPoints,dirtoo):
    for i in range(numPoints[0],numPoints[1]):
        folderr = os.path.join(dirtoo, str(i).zfill(3))
        lanalysis = os.path.join(folderr, 'analysis.txt')
        if os.path.exists(folderr) == False:
            os.mkdir(folderr)
        into = open(lanalysis, 'w')
        into.write(header)
        into.close()
        if saveimgs == True:
            folderb = os.path.join(folderr, 'brightBoxes')
            foldero = os.path.join(folderr, 'edgeDetection')
            folderf = os.path.join(folderr, 'captures')
            foldertm = os.path.join(folderr, 'trackedMolecule')
            folderp = os.path.join(folderr, 'peaks')
            folderpl = os.path.join(folderr, 'plots')
            if os.path.exists(folderb) == False:
                os.mkdir(folderb)
                os.mkdir(foldero)
                os.mkdir(folderf)
                os.mkdir(foldertm)

In [7]:
def makeFolder(i,dirtoo):
    folderr = os.path.join(dirtoo, str(i).zfill(3))
    if os.path.exists(folderr) == False:
        os.mkdir(folderr)
    if saveimgs == True:
        folderb = os.path.join(folderr, 'brightBoxes')
        foldero = os.path.join(folderr, 'edgeDetection')
        folderf = os.path.join(folderr, 'captures')
        foldertm = os.path.join(folderr, 'trackedMolecule')
        folderp = os.path.join(folderr, 'peaks')
        folderpl = os.path.join(folderr, 'plots')
        if os.path.exists(folderb) == False:
            os.mkdir(folderb)
            os.mkdir(foldero)
            os.mkdir(folderf)
            os.mkdir(foldertm)

In [8]:
def findBackground(img):
    avgz_fit = np.zeros(img.shape, np.float64)
    len_x = img.shape[0]
    len_y = img.shape[1]
    z = img.flatten()
    z = z.reshape((len(z),1))
    y = np.array([ i for i in range(len_y) ]*len_x)
    x = np.array([ [i]*len_y for i in range(len_x)  ])
    x = x.flatten()
    xx = np.power(x, 2)
    yy = np.power(y, 2)
    xy = x*y
    ones = np.ones(len(z))
    H = np.array([ xx, yy, xy, x, y, ones ])
    H = H.T
    c = np.dot(H.T, H)
    c = np.linalg.inv(c) 
    c = np.dot(c, H.T)
    c = np.dot(c, z)
    z_fit = np.dot(H, c)
    z_fit = z_fit.reshape((len_x, len_y)).astype(np.uint16)
    return(z_fit)

In [9]:
def findObjects(img, thrval):
    mean = np.mean(img)
    saliency = np.power(img - mean,2)
    dx = ndimage.sobel(saliency, 0)  # horizontal derivative
    dy = ndimage.sobel(saliency, 1)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255 / np.max(mag)  # normalize
    binary_mask = mag >= thrval
    clean_mask = ndimage.morphology.binary_fill_holes(binary_mask).astype(np.uint8)
    return(clean_mask,mag.astype(np.uint8),binary_mask.astype(np.uint8))

In [10]:
def describeContour(img,thr,point):
    contours = cv2.findContours(thr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    lowdis = -999
    for icont in contours[0]:
        bdis = cv2.pointPolygonTest(icont,point, True)
        if bdis > lowdis:
            lowdis = bdis
            cont = icont
    if lowdis == -999:
        return(False)
    area = cv2.contourArea(cont)
    per = cv2.arcLength(cont, True)
    M = cv2.moments(cont)
    x = (int(M['m10']/M['m00']) if M['m00'] > 0 else 0)
    y = (int(M['m01']/M['m00']) if M['m00'] > 0 else 0)
    mask = np.zeros(img.shape,np.uint8)
    cv2.drawContours(mask, [cont], -1, 255,-1)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(img, mask)
    mean = cv2.mean(img, mask)[0]
    return([cont,x,y,area,per,mean,max_loc[1],max_loc[0],max_val,mask])

In [11]:
def clickPoint(event, x, y, flags, param):
    global refPt
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [y,x]

In [12]:
def onFrame(imgnum, img):
    global points
    global refPt
    global stackedPoints
    eimg = my_convert_16U_2_8U(img)
    pct = [x[imgnum][1] for x in  stackedPoints]
    for p in pct:
        cv2.circle(eimg, (p[1],p[0]), 7, (255,255,255),-1)
    while True:
        refPt = []
        cv2.imshow("image", eimg)
        cv2.setMouseCallback("image", clickPoint)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()
        if (k==esc or k==left or k==right or k==enter):
            if k==enter and refPt != []:
                points.append([imgnum, refPt])
            break
    return(k)

In [13]:
def setGain(img):
    ratios = [1,1.5,2,4,8,12,16,20,24,100]
    n = 0
    while True:
        r = ratios[n]
        maxm = int(65535/r)
        dimg = img.copy()
        dimg[dimg>maxm]=65535
        if np.amin(dimg) >= maxm:
            break
        eimg = my_convert_16U_2_8U(dimg*r)
        cv2.imshow("image", eimg)
        cv2.setMouseCallback("image", clickPoint)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()
        if k==enter or n == len(ratios)-1:
            n+=1
        elif k==esc:
            break
    return(ratios[n],eimg)

In [14]:
def watershedFindObjects(img, low, high, smallest):
    sobeco = filters.sobel(img)
    markco = np.zeros_like(img)
    markco[const<low] = 1
    markco[const>high] = 2
    segmco = watershed(sobeco, markco)
    segmco = ndimage.binary_fill_holes(segmco - 1)
    labelco, labels = ndimage.label(segmco)
    sizes = np.bincount(labelco.ravel())
    mask_sizes = sizes > smallest
    mask_sizes[0] = 0
    segmco_cleaned = mask_sizes[labelco]
    return(segmco_cleaned)

In [15]:
def dilateErode(img,ndilate,nerode):
    ddisk = morphology.disk(ndilate)
    edisk = morphology.disk(nerode)
    return(morphology.binary_erosion(morphology.binary_dilation(img,footprint=ddisk), footprint=edisk))

In [16]:
def getDistance(pi,pt):
    xdis = abs(pi[1] - pt[1])
    ydis = abs(pi[0] - pt[0])
    return(sqrt(xdis*xdis + ydis*ydis))

In [17]:
def makeLabeledImage(img,points):
    font = cv2.FONT_HERSHEY_SIMPLEX
    dimg = my_convert_16U_2_8U(img)
    cimg = cv2.merge([dimg,dimg,dimg])
    for n, y, x in points:
        cvy = len(img)-y
        cv2.putText(cimg,str(n),(int(x),int(y)),font,1,[255,0,0])
    return(cimg)

In [18]:
def adaptiveFlatten(flist):
    iterables = ["<class 'tuple'>","<class 'list'>","<class 'numpy.ndarray'>"]
    alist = flist.copy()
    while True:
        blist = []
        br = True
        for tit in alist:
            if str(type(tit)) in iterables:
                if len(tit) >1:
                    br = False
                    break
        if br == True:
            break
        for eone in alist:
            if str(type(eone)) in iterables:
                for item in list(eone):
                    blist.append(item)
            else:
                blist.append(eone)
        alist = blist.copy()
    return alist

In [19]:
'''Ask for picture location'''
print("\n please choose video folder!")
dirtouse = askdirectory(title = "Select foldername", initialdir = '/home/edwin/Desktop/')
#dirtouse = "/home/edwin/Downloads/6"
print(dirtouse)

tifs = []
imgs = []
vidnum = -1

'''Move through all files in directory, saving file names and images if they're tifs'''
for filename in sorted(os.listdir(dirtouse)):
    if filename.endswith('.tif'):
        num = filename[-8:-4]
        if num == "0000":
            tifs.append([])
            imgs.append([])
            vidnum+=1
        tifs[vidnum].append(filename)
        pictouse = os.path.join(dirtouse,filename)
        img = io.imread(pictouse,-1)
        imgs[vidnum].append(img)

length = len(imgs)
print(length)


 please choose video folder!
C:/Users/Eamon/Desktop/6t2/6
6


In [20]:
print('Segmenting images, this may take some time')
'''Move through videos'''
pdata = pd.DataFrame()
segm_imgs = []
segm_consts = []
consts = []
for vidn in range(len(tifs)):
    print(str(vidn+1)+"/"+str(length))
    video = imgs[vidn]
    medianv = np.median(filters.gaussian(img.astype(np.uint16), preserve_range=True))
    const = video[0].copy()
    for img in video[1:]:
        const = np.minimum(const,img)
    segmco = watershedFindObjects(filters.gaussian(const),medianv,medianv+5000,2)
    segmco = dilateErode(segmco,2,2)
    newff = dirtouse+str(vidn)
    if not os.path.exists(newff):
        os.mkdir(newff)
    io.imsave(os.path.join(dirtouse+str(vidn),"constsegment.tif"), segmco)
    io.imsave(os.path.join(dirtouse+str(vidn),"const.tif"), const)
    consts.append(const)
    segm_consts.append(segmco)
    segm_imgs.append([])
    for i in range(len(video)):
        img = video[i]
        imgname = tifs[vidn][i][:-4]
        segmimg = watershedFindObjects(filters.gaussian(img),medianv,medianv+5000,2)
        segmimg = dilateErode(segmimg,2,2)
        io.imsave(os.path.join(dirtouse+str(vidn),str(i)+"_segment.tif"), segmimg)
        segm_imgs[vidn].append(segmimg)

Segmenting images, this may take some time
1/6
2/6
3/6
4/6
5/6
6/6


In [21]:
'''Trying to figure out if new stuff appears. 
For each object in the constant image, get a label. 
Match objects in every image to the labeled constant objects.
Look at objects that do not fit in.
'''
for vidn in range(length):
    print(str(vidn+1)+"/"+str(length))
    video = imgs[vidn]
    seg_vid = segm_imgs[vidn]
    const = consts[vidn]
    seg_const = segm_consts[vidn]
    label_const = measure.label(seg_const)
    constroids = []
    for region in measure.regionprops(label_const, intensity_image=const):
        constroids.append([[region.label-1,region.centroid_weighted]])
    points = []
    for c in constroids:
        points.append([c[0][0],c[0][1][0],c[0][1][1]])
    segmlabel = makeLabeledImage(seg_const.astype(np.uint8)*255,points)
    io.imsave(os.path.join(dirtouse+str(vidn),"labeled_const.tif"),segmlabel)
    outliercentroids = []
    orderedoutliercentroids = []
    for imgn in range(len(video)):
        img = video[imgn]
        segimg = seg_vid[imgn]
        label_img = measure.label(segimg)
        imcentroids = []
        for region in measure.regionprops(label_img, intensity_image=img):
            imcentroids.append([region.label-1,region.centroid_weighted,
                                region.axis_major_length,region.axis_minor_length,np.sum(region.image_intensity),
                                region.moments_hu, region.equivalent_diameter_area,region.area])
        for con in range(len(constroids)):
            constroid = constroids[con]
            neardis = 1000
            for imn in range(len(imcentroids)):
                centroid = imcentroids[imn]
                centroidpt = centroid[1]
                constroidpt = constroid[0][1]
                dis = getDistance(centroidpt,constroidpt)
                if dis < neardis:
                    neardis = dis
                    nearn = imn
            if neardis < 5:
                pcent = imcentroids.pop(nearn)
                constroids[con].append(adaptiveFlatten([imgn,pcent]))
            if len(imcentroids)==0:
                break
        outliercentroids.append(imcentroids)
    kl=[]
    for cobjb in constroids:
        for cobj in cobjb:
            if len(cobj) == 2:
                i = cobj[0]
                continue
            kl.append(adaptiveFlatten([i,cobj]))
    loc = os.path.join(dirtouse,str(vidn)+"trackedMolecules.csv")
    with open(loc, "w") as f:
        write = csv.writer(f)
        write.writerow(["ConstantLabel","ImageNumber","ImageLabel","CentroidY","CentroidX","MajorAxis",
                        "MinorAxis","TotalIntensity","M1","M2","M3","M4","M5","M6","M7","Diameter","Area"])
        write.writerows(kl)
    imgoutlist = list(range(len(outliercentroids)-1))
    for imgi in imgoutlist:
        imgouts = outliercentroids[imgi]
        if imgouts == []:
            continue
        for outlier in imgouts:
            brkthesignal = False
            if outlier[0] == -1:
                continue
            op = outlier[1]
            for constroid in constroids:
                cp = constroid[0][1]
                dis = getDistance(op,cp)
                if dis < 5:
                    brkthesignal = True
                    break
            if brkthesignal == True:
                continue
            orderedoutliercentroids.append([adaptiveFlatten([imgi,outlier])])
            for testi in imgoutlist[imgi+1:]:
                if testi == []:
                    continue
                testouts = outliercentroids[testi]
                for testouti in range(len(testouts)):
                    tp = testouts[testouti][1]
                    dis = getDistance(op,tp)
                    if dis < 5:
                        pcent = outliercentroids[testi][testouti].copy()
                        orderedoutliercentroids[-1].append(adaptiveFlatten([testi,pcent]))
                        outliercentroids[testi][testouti][0] = -1
    loc = os.path.join(dirtouse,str(vidn)+"outlierMolecules.csv")
    oocl=[]
    for oocset in orderedoutliercentroids:
        if oocset == []:
            continue
        for ooc in oocset:
            if ooc == []:
                continue
            oocl.append(ooc)
    with open(loc, "w") as f:
        write = csv.writer(f)
        write.writerow(["ImageNumber","ImageLabel","CentroidY","CentroidX","MajorAxis","MinorAxis",
                        "TotalIntensity","M1","M2","M3","M4","M5","M6","M7","Diameter","Area"])
        write.writerows(oocl)

1/6
2/6
3/6
4/6
5/6
6/6


In [22]:
'''
vidmeds=[]
for vidn in range(length):
    video = imgs[vidn]
    meds=[]
    for i in range(len(video)):
        img = video[i]
        gimg = filters.gaussian(img.astype(np.uint16), preserve_range=True)
        meds.append(np.median(gimg))
    vidmeds.append(sum(meds)/len(meds))


img = np.zeros([1000,1000])
cv2.circle(img,(150,300),7, (255,255,255),-1)
points = [[0,500,50],[1,600,700],[2,850,150]]

plt.imshow(seg_const.astype(np.uint8))
print(seg_const.astype(np.uint8))

points = []
for c in constroids:
    points.append([c[0][0],c[0][1][0],c[0][1][1]])

makeLabeledImage(seg_const.astype(np.uint8)*255,points)
cv2.waitKey(0)
cv2.destroyAllWindows()


imcentroids.append([region.label-1,region.centroid_weighted,region.num_pixels,
                                region.axis_major_length,region.axis_minor_length,np.sum(region.image_intensity),
                                region.moments_hu, region.equivalent_diameter_area])


subtracts = []
for vidn in range(len(tifs)):
    segvid = segm_imgs[vidn]
    subtracts.append([])
    const = segm_consts[vidn]
    for i in range(len(segvid)):
        segmimg = segvid[i]
        imgsub = segmimg ^ const
        subtracts[vidn].append(imgsub)
        io.imsave(os.path.join("/home/edwin/Desktop/6Test/"+str(vidn),str(i)+"_imgsub.tif"), imgsub)

    #Analyzes molecules
    panalysis = pd.DataFrame()
    for n, point in enumerate(points):
        #find centroid and objects to save
        brightBox=getBrightBox(img,point,outL)
        gbgdbBox=getBrightBox(gbgdImg,point,outL)
        foundobjsn,edges,foundobjs = findObjects(gbgdbBox[0],10)
        cont = describeContour(brightBox[0],foundobjsn,(point[1]-brightBox[1],point[0]-brightBox[2]))
        outMean = cv2.mean(brightBox[0],mask=(255 - cont[9]))[0]
        p = [int(cont[2])+brightBox[2],int(cont[1])+brightBox[1]]

        #save files
        cv2.circle(eimg, (p[1],p[0]), 20, (255,255,255),2)
        miss = brightBox[4]  
        bboo = cv2.copyMakeBorder(brightBox[0], miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
        ftoo = cv2.copyMakeBorder(cont[-1], miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
        otoo = cv2.copyMakeBorder(edges, miss[0], miss[1], miss[2], miss[3], cv2.BORDER_CONSTANT)
        bsubfolder = os.path.join(subfolder,'brightBoxes')
        osubfolder = os.path.join(subfolder,'captures')
        fsubfolder = os.path.join(subfolder,'edgeDetection')
        esubfolder = os.path.join(subfolder,'trackedMolecule')
        bbooName = os.path.join(bsubfolder,(str(n)+'_brightBox.tif'))
        otooName = os.path.join(osubfolder,(str(n)+'_edges.tif'))
        ftooName = os.path.join(fsubfolder,(str(n)+'_capture.tif'))
        eimgName = os.path.join(esubfolder,(str(n)+'_point.tif'))
        cv2.imwrite(bbooName, bboo)
        cv2.imwrite(otooName, otoo)
        cv2.imwrite(ftooName, ftoo)
        cv2.imwrite(eimgName, eimg)
        pointNumber = str(n+1).zfill(3)
        Aimg = io.imread(bbooName)
        Athr = io.imread(ftooName)
        lAthr = measure.label(Athr)
        props = measure.regionprops_table(lAthr,intensity_image=Aimg,
                                        properties=['bbox', 'bbox_area', 'centroid', 
                                                    'convex_area', 'eccentricity', 'equivalent_diameter', 
                                                    'inertia_tensor', 'inertia_tensor_eigvals', 
                                                    'label', 'local_centroid', 'major_axis_length', 
                                                    'max_intensity', 'mean_intensity', 
                                                    'min_intensity', 'minor_axis_length', 'moments', 
                                                    'moments_central', 'moments_hu', 
                                                    'moments_normalized', 'orientation', 'perimeter', 
                                                    'solidity', 'weighted_centroid', 
                                                    'weighted_local_centroid', 'weighted_moments', 
                                                    'weighted_moments_central', 'weighted_moments_hu', 
                                                    'weighted_moments_normalized'])
        props['x'] = p[0]
        props['y'] = p[1]
        props['pointr'] = pointNumber
        panalysis = panalysis.append(pd.DataFrame(props).dropna(axis=1))
        props['pointr'] = pointNumber
        pdata = pdata.append(pd.DataFrame(props).dropna(axis=1))
    panalysis.to_csv(os.path.join(subfolder,'analysis.csv'))
pdata.to_csv(os.path.join(dirtouse,'totalAnalysis.csv'))
'''

'\nvidmeds=[]\nfor vidn in range(length):\n    video = imgs[vidn]\n    meds=[]\n    for i in range(len(video)):\n        img = video[i]\n        gimg = filters.gaussian(img.astype(np.uint16), preserve_range=True)\n        meds.append(np.median(gimg))\n    vidmeds.append(sum(meds)/len(meds))\n\n\nimg = np.zeros([1000,1000])\ncv2.circle(img,(150,300),7, (255,255,255),-1)\npoints = [[0,500,50],[1,600,700],[2,850,150]]\n\nplt.imshow(seg_const.astype(np.uint8))\nprint(seg_const.astype(np.uint8))\n\npoints = []\nfor c in constroids:\n    points.append([c[0][0],c[0][1][0],c[0][1][1]])\n\nmakeLabeledImage(seg_const.astype(np.uint8)*255,points)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n\n\nimcentroids.append([region.label-1,region.centroid_weighted,region.num_pixels,\n                                region.axis_major_length,region.axis_minor_length,np.sum(region.image_intensity),\n                                region.moments_hu, region.equivalent_diameter_area])\n\n\nsubtracts = []\nfor vidn 